In [1]:
import pandas as pd
import hashlib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

N_PARTITIONS = 1000

In [2]:
def id_to_hash(customer_id):
    return int(hashlib.md5(customer_id.encode('utf-8')).hexdigest(), 16)

In [3]:
members = pd.read_csv('/data/churn/members_v3.csv', nrows = 1)
members

transactions = pd.read_csv('/data/churn/all_trans.csv', nrows = 1)
transactions

logs = pd.read_csv('/data/churn/user_logs.csv', nrows = 1)
logs

train = pd.read_csv('/data/churn/all_train.csv', nrows = 1)
train

test = pd.read_csv('/data/churn/sample_submission_v2.csv', nrows = 1)
test

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,20150930,20151101,0


,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,20150513,0,0,0,0,1,1,280.335


,msno,is_churn
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1


,msno,is_churn
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0


In [4]:
id_to_hash(members.loc[0, 'msno'])

209512247756457468966515739358104959027

In [5]:
import os

base_dir = '/data/churn/partitions/'

# Create 1000 directories for each partition
for i in range(1000):
    os.makedirs(base_dir + f'p{i}', exist_ok=True)
    
len(os.listdir(base_dir))

1000

In [6]:
','.join(list(transactions.columns))

'msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel'

## Create Files

In each partition there are 5 csv files. The following code writes the header for each of the five files in each of the `N_PARTITION` partitions.

In [7]:
InteractiveShell.ast_node_interactivity = 'last_expr'

In [8]:
def create_blank_partitions():
    # For each partition create the files with headers
    for i in range(N_PARTITIONS):
        directory = base_dir + f'p{i}/'
        # Create five files
        for file in ['transactions.csv', 'train.csv', 'test.csv', 'members.csv', 'logs.csv']:
            # Write file header as first line
            with open(directory + file, 'w') as f:
                if file == 'transactions.csv':
                    f.write(','.join(list(transactions.columns)))
                elif file == 'train.csv':
                    f.write(','.join(list(train.columns)))
                elif file == 'test.csv':
                    f.write(','.join(list(train.columns)))
                elif file == 'members.csv':
                    f.write(','.join(list(members.columns)))
                elif file == 'logs.csv':
                    f.write(','.join(list(logs.columns)))
                    
    return directory

directory = create_blank_partitions()

In [9]:
os.listdir(directory)

['transactions.csv', 'members.csv', 'test.csv', 'train.csv', 'logs.csv']

### Example of Writing a Row

For each file, we'll iterate over the rows one at a time. The process for handling a row is:

1. Convert the customer id to an integer by hashing
2. Convert the integer to a partition number by modulus dividing by the number of partitions
3. Append the row to the correct partition file

Let's see how this works with a single row.

In [10]:
# Iterate through the dataframe one row at a time
for i, row in members.iterrows():
    # Find the partition by hashing the id
    partition = id_to_hash(row['msno']) % N_PARTITIONS
    # Open the file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        # Write a newline and then the information
        f.write('\n')
        f.write(','.join([str(x) for x in row.values]))
    if i > 1:
        break

In [11]:
pd.read_csv(base_dir + f'p{partition}/members.csv')

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911


Everything looks like it went well with the first attempt. Now we'll run this on a complete dataset.

# Performance of Different Methods

To find out which method is the best, we'll try 4 approaches:

1. df.iterrows(): iterate through the dataframe one row at a time with rows represented as series
2. df.itertuples(): iterate through the dataframe one row at a time with rows represented as tuples
3. df.apply(): iterate through the data one row at a time using `apply`
4. `groupby(partition)` and save each group with `to_csv()`: iterate through the dataframe one partition at a time

The four approaches have different applicability and performance characteristics (see this [Stack Overflow](https://stackoverflow.com/questions/24870953/does-iterrows-have-performance-issues/24871316#24871316) answer). The way to find out which one is the quickest is to try them all. This isn't meant to represent all use cases, so your particular results may vary.

In [12]:
from timeit import default_timer as timer

### Member Information

We'll start off with the members. 

In [13]:
members = pd.read_csv('/data/churn/members_v3.csv')
members.shape

(6769473, 6)

## Iterrows

The first implementation will try using `iterrows`. This is fairly slow because Pandas packages the row as a Pandas series before iteration.

In [ ]:
start = timer()

for i, row in members.iterrows():
    # Find the partition number by hashing the id
    partition = id_to_hash(row['msno']) % N_PARTITIONS
    
    # Open file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        # Write a new line and then data
        f.write('\n')
        f.write(','.join([str(x) for x in row.values]))
        
    if i % 10000 == 0:
        print(f'{100 * round(i / members.shape[0], 2)}% complete. {round(timer() - start)} seconds elapsed.', end = '\r')

end = timer()
print(f'Processing {i} lines took {round(end - start)} seconds using iterrows.')

In [ ]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

## Itertuples

Itertuples should be faster than iterrows because Pandas packages the row as a tuple instead of as a series. 

In [ ]:
_ = create_blank_partitions()

In [ ]:
start = timer()

for i, row in enumerate(members.itertuples()):
    
    # Find the partition number by hashing the id
    partition = id_to_hash(row[1]) % N_PARTITIONS
    
    # Open file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        # Write a new line and then data
        f.write('\n')
        f.write(','.join([str(x) for x in row[1:]]))
        
    if i % 10000 == 0:
        print(f'{100 * round(i / members.shape[0], 2)}% complete. {round(timer() - start)} seconds elapsed.', end = '\r')

end = timer()
print(f'Processing {i} lines took {round(end - start)} seconds using itertuples.')

In [ ]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

In [ ]:
_ = create_blank_partitions()

## Apply

The other operation that could be used is apply. To use `apply`, we write a small function that saves the row and then call apply to the dataframe using `axis = 1` which sends each row to the function.

In [ ]:
def save_row(row, name):
    # Find the partition number by hashing the id
    partition = id_to_hash(row['msno']) % N_PARTITIONS
    
    # Open file for appending
    with open(base_dir + f'p{partition}/{name}.csv', 'a') as f:
        # Write a new line and then data
        f.write('\n')
        f.write(','.join([str(x) for x in row.values]))

In [ ]:
from tqdm import tqdm_notebook
from tqdm import tqdm
tqdm.pandas()

start = timer()
members.progress_apply(save_row, axis = 1, name = 'members')
end = timer()

print(f'Processing {members.shape[0]} rows took {round(end - start)} seconds using apply.')

In [ ]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

In [ ]:
_ = create_blank_partitions()

## Groupby 

Another option is to compute the partitions with the hashing function all at once, groupby the partition, and write the grouped dataframe to disk.

In [ ]:
%%timeit -n 1 -r 3
members['msno'].map(id_to_hash) % 1000

In [ ]:
%%timeit -n 1 -r 3
members['msno'].apply(id_to_hash) % 1000

In [ ]:
members['partition'].value_counts().head()

In [ ]:
start = timer()
members['partition'] = members['msno'].map(id_to_hash) % N_PARTITIONS

# Iteration through grouped partitions
for partition, grouped in members.groupby('partition'):
    
    grouped = grouped.drop(columns = 'partition')
    # Open file for appending
    with open(base_dir + f'p{partition}/members.csv', 'a') as f:
        f.write('\n')
        grouped.to_csv(f, header = False, index = False)
        
end = timer()
print(f'Processing {members.shape[0]} rows took {round(end - start)} seconds using groupby.')

In [ ]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

It looks as if calculating the partition, grouping the data, and writing each group to a file is the quickest way to go.

## Reusable Hashing Dataframe Function

To make the process reusable, we'll write a function that does this for us. It will take in a dataframe, a name for the file to save the data to, and an optional progress argument. The function will map the customer id (`msno`) to a partition number using the hash modulo the number of partitions, group the dataframe by the partition, and write the grouped dataframe to the appropriate directory. 

In [ ]:
members = pd.read_csv('/data/churn/members_v3.csv', nrows = 1)
transactions = pd.read_csv('/data/churn/all_trans.csv', nrows = 1)
logs = pd.read_csv('/data/churn/user_logs.csv', nrows = 1)
train = pd.read_csv('/data/churn/all_train.csv', nrows = 1)
test = pd.read_csv('/data/churn/sample_submission_v2.csv', nrows = 1)

_ = create_blank_partitions()

In [ ]:
pd.read_csv('/data/churn/partitions/p999/members.csv').head()

In [ ]:
def partition_by_hashing(df, name, progress = None):
    """Partition a dataframe into N_PARTITIONS by hashing the id.
    
    Params
    --------
        df (pandas dataframe): dataframe for partition. Must have 'msno' column.
        name (str): name of dataframe. Used for saving the row data.
        progress (int, optional): number of rows to be processed before displaying information.
                                  Defaults to None
                                  
    Returns:
    --------
        Nothing returned. Dataframe is saved one line at a time as csv files to the N_PARTITIONS 
    """
    start = timer()
    
    # Map the customer id to a partition number
    df['partition'] = df['msno'].map(id_to_hash) % N_PARTITIONS
    
    # Iterate through one row at a time
    for partition, grouped in df.groupby('partition'):
        
        # Don't need to save the partition column
        grouped = grouped.drop(columns = 'partition')
        
        # Open file for appending
        with open(base_dir + f'p{partition}/{name}.csv', 'a') as f:
            # Write a new line and then data
            f.write('\n')
            grouped.to_csv(f, header = False, index = False)
            
        # Record progress every `progress` steps
        if progress is not None:
            if partition % progress == 0:
                print(f'{100 * round(partition / N_PARTITIONS, 2)}% complete. {round(timer() - start)} seconds elapsed.', end = '\r')
    
    end = timer()
    if progress is not None:
        print(f'\n{df.shape[0]} rows processed in {round(end - start)} seconds.')

In [ ]:
members = pd.read_csv('/data/churn/members_v3.csv')
partition_by_hashing(members, name = 'members', progress = 10)

In [ ]:
pd.read_csv(base_dir + f'p{partition}/members.csv').head()

## Training Data

Now we can use this function to partition the training data.

In [ ]:
train = pd.read_csv('/data/churn/all_train.csv')
partition_by_hashing(train, name = 'train', progress = 10)

In [ ]:
pd.read_csv(base_dir + f'p{partition}/train.csv').head()

## Testing Data

The nice thing about a function is we can keep applying it, changing only the arguments! 

In [ ]:
test = pd.read_csv('/data/churn/sample_submission_v2.csv')
partition_by_hashing(test, name = 'test', progress = 10)
pd.read_csv(base_dir + f'p{partition}/test.csv').head()

## Transactional Data

The second to last dataset are the customer transactions.

In [ ]:
transactions = pd.read_csv('/data/churn/all_trans.csv')
partition_by_hashing(transactions, name = 'transactions', progress = 10)
pd.read_csv(base_dir + f'p{partition}/transactions.csv').head()

## User Log Data

The final dataset cannot be pass directly into the function because of the size. Instead, we can read it in a chunk at a time using Pandas and apply the function to each chunk.

In [ ]:
print(os.stat('/data/churn/user_logs.csv').st_size / 1e9)
print(os.stat('/data/churn/user_logs_v2.csv').st_size / 1e9)

The second user logs can be processed in the previous manner, but we'll use the same chunking framework for both.

In [ ]:
chunksize = 1e6
start = timer()

for chunk in pd.read_csv('/data/churn/user_logs_v2.csv', chunksize = chunksize):
    partition_by_hashing(chunk, name = 'logs', progress = 10)

end = timer()
print(f'\nOverall time: {round(end - start)} seconds.')

In [ ]:
pd.read_csv(base_dir + f'p{partition}/logs.csv').head()

In [ ]:
chunksize = 1e7

start = timer()

for i, chunk in enumerate(pd.read_csv('/data/churn/user_logs.csv', chunksize = chunksize)):
    partition_by_hashing(chunk, name = 'logs', progress = None)
    
    if (i + 1) % 10 == 0:
        print(f'{i * chunksize} rows process.')
    
end = timer()
print(f'\nOverall time: {round(end - start)} seconds.')

In [ ]:
pd.read_csv(base_dir + f'p{partition}/logs.csv').tail()